<a href="https://colab.research.google.com/github/Jimin980921/DeepLearning/blob/master/MNIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


tensorflow는 계산을 위해 고효율의 c++백엔드를 사용함. 
tensorflow는 session을 사용함.

tensorflow는 먼저 그래프를 구성하고, 그래프를 세션을 통해 실행함.


tensorflow코드를보다 유연하게 작성할 수 있게 해주는 InteractiveSession클래스를 사용함.
이 클래스는 계산 그래프와 실행작업을 분리시켜줌.

In [0]:
sess=tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# **계산그래프**
python에서 효율적인 수치계산을 위해,tensorflow는 상호작용하는 연산을 그래프로 묶어 그 전체를 python바깥에서 실행시키는 방법을 사용.


# **소프트맥스 회귀(softmax regression)모델 구성**
다층 계층의 합성곱 신경망을 구성할 것임.

# **플레이스홀더(Placeholder)**
x,y_에 특정한 값을 부여한 것이 아님.

tensorflow가 계산을 실행할때, 값을 넣어줄 자리가 placeholder이다.

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])  
#부동소수점실수,2차원 tensor. 784는 28*28 크기를 가진 mnist이미지를 한줄로 펼친 크기
y_ = tf.placeholder(tf.float32, shape=[None, 10])   
#2차원 tensor,mnist이미지의 숫자 클래스(one-hot벡터로 나타냄.)

# **변수**
가중치(w),편향(b)
tensorflow는 이러한 변수를 다루기위해 Variable을 제공

Variable이란 tensorflow의 계산 그래프안에 있는 값(계산에의해 변경되거나, 계산에 사용될 수 있음.)


In [0]:
#Variable들은 세션이 시작되기전에 초기화되어야함.
W = tf.Variable(tf.zeros([784,10]))
#이미지벡터크기 784, 출력숫자클래스 10
b = tf.Variable(tf.zeros([10]))
#10차원 벡터

In [0]:
sess.run(tf.global_variables_initializer())
#Variable들 각각에 대해 미리 지정된 초기값을 넣어주는 역할

# **클래스 예측 및 비용 함수**

벡터화된 이미지인 x를 가중치 행렬인 W와 곱하고, 여기에 편향 b를 더한 뒤, 각각의 클래스에 대한 소프트맥스 함수의 결과를 계산

In [0]:
y=tf.nn.softmax(tf.matmul(x,W)+b)

In [0]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#손실함수는 크로스-엔트로피함수 사용
#reduce_mean은 사용된 이미지들 각각에서 계산된 합의 평균을 구해주는 함수. y의 2번째 차원(reduction_indices=[1])의 원소들을 합함.

# **모델 훈련시키기**
tensorflow는 전체 계산 그래프의 정보가 입력되어있음. 

라이브러리가 자동으로 미분을 통해 각각 변수에대한 손실함수의 그레디언트를 계산

경사하강법을 이용하여 크로스 엔트로피를 최소화할 것임.

In [0]:
train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
#학습속도 0.5
#반환된 train_step은 실행되었을때, 경사하강법을 통해 각각의 매개변수 변화시킴. 
#모델을 훈련시키려면 반복해서 실행하면됨.

In [0]:
for i in range(1000):
  batch=mnist.train.next_batch(50)
  #50개의 다음 훈련데이터 샘플 추출
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})
  #train_step을 실행하여 텐서x와 y_에 훈련데이터 넣어줌.
  #feed_dict는 계산 그래프안에 어떤 텐서도 변경가능.


# **모델 평가**
tf.argmax로 한 차원에서 가장 큰 값의 인덱스를 반환해줌.

tf.argmax(y,1)은 모델이 입력받고 가장 그럴듯하다고 생각한 레이블이고,

tf.argmax(y_,1)은 실제레이블


In [0]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
#불리언으로 이루어진 리스트 반환함. 

In [0]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#부동소수점으로 형변환하면 예측결과정확도나옴. 

In [0]:
print(accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

0.8823


# **다중 계층 합성곱 신경망**



In [0]:
#매번 모델 만들때매다 반복
def weight_variable(shape):
  initial=tf.truncated_normal(shape,stddev=0.1)
  #stddev는 표준편차를 줌.
  return tf.Variable(initial)

def bias_variable(shape):
  initial=tf.constant(0.1,shape=shape)
  #죽은 뉴런을 방지하기위해 편향을 작은양수(0.1)로 초기화함.
  return tf.Variable(initial)

# **합성곱(convolution), 풀링(pooling)**

합성곱:커널이동하면서 계산하는 것.

풀링:특성 맵을 스캔하며 최댓값을 고르거나 평균값을 계산하는 것.

strides:미끄러지는 배열의 간격을 조절하는 것.

padding:원본배열의 양 끝에 빈 원소를 추가하는 것.



In [0]:
def conv2d(x,W):
  return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME") 
  #SAME은 원본배열길이와 출력배열길이를 같게 하겠다는 의미

def max_pool_2X2(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
  #ksize는 풀링의 크기

# **첫번째 합성곱 계층**
합성곱계층에서는 5*5의 윈도우(patch)크기를 가지는 32개의 필터를 사용함

[5,5,1,32]

5,5는 윈도우의크기

1은 입력채널의 수

32는 출력채널의 수(얼마나 많은 특징을 사용할 것인가)



In [0]:
w_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])

In [0]:
x_image=tf.reshape(x,[-1,28,28,1])
#이미지를 입력하려면, x를 4차원텐서로 reshape해야함.
#28 28은 이미지의 가로,세로길이, 1은 컬러채널의 갯수

In [0]:
h_conv1=tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
#합성곱 적용후, relu함수(0보다 크면 그대로통과, 0보다 작으면 0)로 활성화
h_pool1=max_pool_2X2(h_conv1)
#출력값을 구하기위해 마지막으로 맥스 풀링(최대값출력)을 적용

# **두번째 합성곱 계층**
5*5 윈도우(patch)에 64개 필터 사용

In [0]:
w_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])

In [0]:
h_conv2=tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2) 
#첫번째 합성곱 계층의 결과(h_pool1)를 대입)
h_pool2 = max_pool_2X2(h_conv2)

# **완전 연결 계층**

두번째 계층을 거친 뒤 이미지는 7*7로 줄어듬.(?)

1024개의 뉴런을 연결되는 완전 연결 계층 구성

7*7의 이미지배열을 reshpae해야하며, 완전 연결 계층에 맞는 가중치 행렬과 편향행렬을 구해야함.

완전연결계층 끝에 relu함수 써야함.


In [0]:
w_fc1=weight_variable([7*7*64,1024])  #가중치
b_fc1=bias_variable([1024])   #편향

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])  #입력 reshape
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

# **드롭아웃**

오버피팅을 방지하기위해 드롭아웃 적용

드롭아웃이되지 않을 확률을 저장하는 placeholder만듬.(훈련에만 사용되고, 테스트에는 사용되지 않도록 만들 수 있음)

tensorflow의 tf.nn.dropout은 뉴런출력을 자동으로 스케일링하므로, 추가 스케일링 필요없음.


In [0]:
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# 최종 소프트맥스 계층

In [0]:
w_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

# **모델 훈련 및 평가**

adam최적화 알고리즘 사용

In [0]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
#크로스 엔트로피 손실함수
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
#부울 값으로 이루어진 리스트.y_conv는 최종 소프트맥스 계층의 출력
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#이 값을 부동소수점 값으로 변환한 후 평균을 계산
sess.run(tf.global_variables_initializer())
#초기화

for i in range(20000):
  batch=mnist.train.next_batch(50)
  #다음 배치 
  if i%100==0:
    train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
    #드롭아웃이되지 않을 확률을 저장하는 placeholder=keep_prob
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  #train_step을 거침(아담). 최적화 결과 찾음.
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.88
step 400, training accuracy 0.84
step 500, training accuracy 0.96
step 600, training accuracy 0.92
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.92
step 1100, training accuracy 0.98
step 1200, training accuracy 1
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 0.96
step 1700, training accuracy 0.94
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 0.92
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.98
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, 